In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import ee
import geemap
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [11]:
polygon = ee.Geometry.Rectangle([110.961135, 32.444885,111.736092, 33.042745])
roi = ee.FeatureCollection(polygon)
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,9)

# roi = Map.user_roi
# print(roi.getInfo())

In [12]:
# AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7
def AWEIsh(image):
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    return image.addBands(awei)

# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

visParams = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

In [13]:
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2020-01-01', '2020-12-31') \
    .map(maskL8sr) \
    .map(AWEIsh)

In [14]:
#计算水体识别为水体的个数
water_layer = landsat_images.map(lambda i : i.select('AWEI').gt(0)).sum()
# 计算有效像元的个数
validPixel_layer = landsat_images.count().select('B3').clip(roi).rename('count')
vis = {'bands': ['count'], 'palette': ['ffffff', 'dd09ec'], 'min': 0.0, 'max': 35.0, 'opacity': 1.0}
Map.addLayer(validPixel_layer,vis,'validPixel')
waterfrequency = water_layer.select('AWEI').divide(validPixel_layer.select('count')).select('AWEI').rename('frequency')
permanentwater = waterfrequency.gte(0.75).selfMask()

Map.addLayer(permanentwater,{'palette':['green']},"permanent water")

In [15]:
geemap.ee_export_image_to_drive(permanentwater, description='danjiangkou_permanent_water', folder='danjiangkou', region=roi.geometry(), scale=30)

Exporting danjiangkou_permanent_water ...


In [ ]:
selector = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','mNDWI','ndvi_mndwi','AWEI','waterclass']
geemap.ee_export_vector_to_drive(points_collection, description='TrainingPoints_2018', folder='wuhan', file_format='shp', selectors=selector)